In [1]:
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook,save
from sklearn.cluster import MiniBatchKMeans
from sklearn import metrics
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
import lda
import matplotlib.pyplot as plt
from bokeh.io import push_notebook
from sklearn.decomposition import LatentDirichletAllocation

In [2]:
post = pd.read_pickle('/Users/boyaliu/Documents/UCD/STA 208/final project/qpost_df_new.pkl')

In [3]:
post.head()

,Body,ViewCount,ClosedDate,ParentID,CommentCount,AnswerCount,AcceptedAnswerId,Score,OwnerDisplayName,Title,...,SpaceCnt,TagCnt,Sentimental_Polarity,Sentimental_Subjectivity,TitleLen,UrlCnt,CleanBody,CleanTitle,Body_rm,Title_rm
0,<p>How should I elicit prior distributions fro...,1850.0,None,NaN,1,5.0,15.0,31,None,Eliciting priors from experts,...,15,3,0.133333,0.133333,29,0,elicit prior distribution expert fit bayesian ...,elicit prior expert,How should I elicit prior distributions from e...,Eliciting priors from experts
1,<p>In many different statistical methods there...,15519.0,None,NaN,1,7.0,59.0,26,None,What is normality?,...,25,2,0.000000,0.000000,18,0,statistical method assumption normality normal...,normality,In many different statistical methods there is...,What is normality?
2,<p>What are some valuable Statistical Analysis...,5162.0,None,NaN,4,19.0,5.0,63,None,What are some valuable Statistical Analysis op...,...,36,2,0.021875,0.596875,65,0,valuable statistical analysis open source proj...,valuable statistical analysis open source project,What are some valuable Statistical Analysis op...,What are some valuable Statistical Analysis op...
3,<p>I have two groups of data. Each with a dif...,15443.0,None,NaN,2,5.0,135.0,15,None,Assessing the significance of differences in d...,...,90,2,0.129808,0.552885,58,0,group data distribution multiple variable dete...,assess significance difference distribution,I have two groups of data. Each with a differ...,Assessing the significance of differences in d...
5,"<p>Last year, I read a blog post from <a href=...",65657.0,None,NaN,6,17.0,NaN,226,None,The Two Cultures: statistics vs. machine learn...,...,204,1,-0.150000,0.300000,50,4,year read blog post brendan connor entitle sta...,culture statistic machine learning,"Last year, I read a blog post from Brendan O'C...",The Two Cultures: statistics vs. machine learn...


In [3]:
body_title_process = post.CleanBody + ' ' + post.CleanTitle

In [4]:
body_title_process[1]

'statistical method assumption normality normality normality normality'

In [5]:
tag = post.Tags.map(lambda i: re.sub('<|>', ' ', i))

In [16]:
tag

0                             bayesian  prior  elicitation 
1                                 distributions  normality 
2                                    software  open-source 
3                  distributions  statistical-significance 
5                                         machine-learning 
6                    dataset  sample  population  teaching 
7                                                    humor 
9           scales  measurement  ordinal  interval  likert 
10                            multivariable  interpolation 
16            anova  chi-squared  generalized-linear-model 
20                         forecasting  population  census 
21                                   bayesian  frequentist 
22                                 distributions  pdf  cdf 
24                modeling  time-series  finance  software 
25                      standard-deviation  basic-concepts 
29         algorithms  hypothesis-testing  random-variab...
30         hypothesis-testing  t-test  p

In [6]:
tags =tag.map(lambda i:  " ".join([word for word in word_tokenize(i)]))

In [6]:
tags

0                                bayesian prior elicitation
1                                   distributions normality
2                                      software open-source
3                    distributions statistical-significance
5                                          machine-learning
6                        dataset sample population teaching
7                                                     humor
9                scales measurement ordinal interval likert
10                              multivariable interpolation
16               anova chi-squared generalized-linear-model
20                            forecasting population census
21                                     bayesian frequentist
22                                    distributions pdf cdf
24                    modeling time-series finance software
25                        standard-deviation basic-concepts
29        algorithms hypothesis-testing random-variable ...
30        hypothesis-testing t-test p-va

In [7]:
body_title_tag = body_title_process + ' ' + tags

In [8]:
# calculate the tf-idf scores
# min_df: When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold
# max_features: build a vocabulary that only consider the top 10000 max_features ordered by term frequency across the corpus.
vectorizer = TfidfVectorizer(min_df=4, max_features = 10000)
vz = vectorizer.fit_transform(tags)

# Text Clustering

## Use gensim library to preprocess the text

In [8]:
from gensim import corpora, models, similarities
from collections import defaultdict

In [20]:
# only contain words that shows more than 1 time

from collections import defaultdict

frequency = defaultdict(int)
for text in tags:
    for word in word_tokenize(text):
        frequency[word] += 1

In [10]:
# create a function for a text only contains words that appear more than once
def text_model(text):
    output = []
    for word in word_tokenize(text):
        if frequency[word] > 1:
            output.append(word)
    return(output)

In [19]:
texts = tags.map(lambda i: text_model(i))

In [21]:
# created to store a special object from the corpora module of the gensim library. 
# This is responsible for indexing the words and assigning them to numbers.
dictionary= corpora.Dictionary(texts)

In [22]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('contoso.mm', corpus)

In [23]:
tfidf = models.TfidfModel(corpus, normalize=True)

In [24]:
# The corpus is now transformed to match the trained TF/IDF model. 
# Before, the numeric representation was based on the bag of words model (how much times each word appears), 
# and after the transformation, the numeric representation will be based on the times each word appears in the 
# whole corpus and scaled by how many individual discussions use that word. 
corpus_tfidf = tfidf[corpus]

In [25]:
# discover topics 
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=5)
corpus_lsi = lsi[corpus_tfidf]
topics = lsi.show_topics()

In [26]:
print(topics)

[(0, '0.703*"regression" + 0.531*"r" + 0.229*"time-series" + 0.186*"logistic" + 0.117*"machine-learning" + 0.102*"correlation" + 0.098*"hypothesis-testing" + 0.093*"multiple-regression" + 0.089*"self-study" + 0.074*"anova"'), (1, '-0.627*"regression" + 0.528*"r" + 0.425*"time-series" + 0.167*"probability" + 0.138*"hypothesis-testing" + -0.119*"logistic" + 0.119*"forecasting" + 0.110*"distributions" + 0.077*"arima" + 0.072*"machine-learning"'), (2, '0.718*"probability" + 0.361*"self-study" + -0.289*"r" + 0.283*"distributions" + 0.256*"hypothesis-testing" + -0.145*"time-series" + 0.131*"mathematical-statistics" + 0.128*"normal-distribution" + 0.110*"statistical-significance" + 0.089*"conditional-probability"'), (3, '-0.730*"time-series" + 0.440*"r" + 0.297*"machine-learning" + -0.182*"hypothesis-testing" + -0.150*"forecasting" + -0.137*"regression" + -0.125*"correlation" + 0.124*"classification" + 0.092*"probability" + -0.090*"arima"'), (4, '-0.657*"hypothesis-testing" + 0.431*"machine-l

## kmeans

In [28]:
for i in range(2,10):
    kmeans_model = MiniBatchKMeans(n_clusters = i, init='k-means++', n_init=1, 
                         init_size=1000, batch_size=1000, verbose=False, max_iter=1000, random_state = 0)
    kmeans = kmeans_model.fit(vz)
    kmeans_clusters = kmeans.predict(vz)
    kmeans_distances = kmeans.transform(vz)
    X = kmeans_model.fit_transform(vz)
    print('for k is', i, 'score is', metrics.silhouette_score(X, kmeans_model.labels_, sample_size=10000))



for k is 2 score is 0.772903201803
for k is 3 score is 0.626117054927
for k is 4 score is 0.597956774256
for k is 5 score is 0.548168879471
for k is 6 score is 0.540534799606
for k is 7 score is 0.51457631307
for k is 8 score is 0.502970259464
for k is 9 score is 0.489731863842


In [9]:
# set number of clusters as 8
num_clusters = 8
kmeans_model = MiniBatchKMeans(n_clusters=num_clusters, init='k-means++', n_init=1, 
                               init_size=1000, batch_size=1000, verbose=False, max_iter=1000, random_state = 0)
kmeans = kmeans_model.fit(vz)
kmeans_clusters = kmeans.predict(vz)
kmeans_distances = kmeans.transform(vz)

In [10]:
sorted_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(num_clusters):
    print("Cluster %d:" % i, end='')
    for j in sorted_centroids[i, :3]:
        print(' %s' % terms[j], end='')
    print()

Cluster 0: testing hypothesis correlation
Cluster 1: regression multiple logistic
Cluster 2: data visualization categorical
Cluster 3: series time forecasting
Cluster 4: value expected hypothesis
Cluster 5: model linear mixed
Cluster 6: learning machine classification
Cluster 7: probability self study


In [11]:
tsne_model = TSNE(n_components=2, verbose=1, random_state=0)
tsne_kmeans = tsne_model.fit_transform(kmeans_distances[:10000])

[t-SNE] Computing pairwise distances...
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Computed conditional probabilities for sample 1000 / 10000
[t-SNE] Computed conditional probabilities for sample 2000 / 10000
[t-SNE] Computed conditional probabilities for sample 3000 / 10000
[t-SNE] Computed conditional probabilities for sample 4000 / 10000
[t-SNE] Computed conditional probabilities for sample 5000 / 10000
[t-SNE] Computed conditional probabilities for sample 6000 / 10000
[t-SNE] Computed conditional probabilities for sample 7000 / 10000
[t-SNE] Computed conditional probabilities for sample 8000 / 10000
[t-SNE] Computed conditional probabilities for sample 9000 / 10000
[t-SNE] Computed conditional probabilities for sample 10000 / 10000
[t-SNE] Mean sigma: 0.000000
[t-SNE] Error after 100 iterations with early exaggeration: 1.301112
[t-SNE] Error after 300 iterations: 1.224848


In [52]:
kmeans_clusters

array([0, 0, 0, ..., 0, 7, 0], dtype=int32)

In [90]:
# create the plot and save it
colormap = np.array([
    "#1f77b4", "#aec7e8", "#ff7f0e", "#ffbb78", "#2ca02c", 
    "#98df8a", "#d62728", "#ff9896", "#9467bd", "#c5b0d5", 
    "#8c564b", "#c49c94", "#e377c2", "#f7b6d2", "#7f7f7f", 
    "#c7c7c7", "#bcbd22", "#dbdb8d", "#17becf", "#9edae5"
])
leg = {0:'testing hypothesis correlation',1:'regression multiple logistic', 
                           2:'data visualization categorical', 3:'series time forecasting',
                             4:'value expected hypothesis', 5:'model linear mixed', 
       6:'learning machine classification',
                             7:'probability self study'}

plot_kmeans = bp.figure(plot_width=900, plot_height=700, title="Stack Overflow Questions (k-means)",
    tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
    x_axis_type=None, y_axis_type=None, min_border=1)

plot_kmeans.scatter(x=tsne_kmeans[:,0], y=tsne_kmeans[:,1], 
                    color=colormap[kmeans_clusters][:10000], 
                    source=bp.ColumnDataSource({
                       "tags": tags[:10000],
                       "cluster": kmeans_clusters[:10000]
                    }))

hover = plot_kmeans.select(dict(type=HoverTool))
hover.tooltips={"tags": "@tags (tags: \"@tags\" - cluster: @cluster)"}
#hover.tooltips={"questions": "@questions (processed: \"@processed\" - cluster: @cluster)"}
save(plot_kmeans, filename = '/Users/boyaliu/Documents/UCD/STA 208/final project/k8.html')



/Users/boyaliu/anaconda/lib/python3.5/site-packages/bokeh/io.py:393: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warnings.warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/Users/boyaliu/anaconda/lib/python3.5/site-packages/bokeh/io.py:398: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warnings.warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


## Latent Dirichlet Allocation (LDA)

In [8]:
# uncover the latent topics in the post
# and then wet're going to use the topic distributions for each document as a measure to group similar tweets together.
# vectorize our data by representing each post as a 10k dimensional vector whose indices correspond to the 
# 10k most frequent terms in our corpus.
cvectorizer = CountVectorizer(min_df=4, max_features=10000, stop_words='english')
cvz = cvectorizer.fit_transform(body_title_tag)

In [9]:
for i in range(1, 10):
    lda_model = lda.LDA(n_topics=i, n_iter = 2000, random_state = 0)
    lda_model.fit(cvz)
    score = lda_model.loglikelihoods_[0]
    print('for i is', i, 'the score is', score)

for i is 1 the score is -29520933.199726492
for i is 2 the score is -32663333.57656804
for i is 3 the score is -34585097.17157614
for i is 4 the score is -36001729.08554066
for i is 5 the score is -37137933.2914421
for i is 6 the score is -38092794.60097875
for i is 7 the score is -38923540.63248336
for i is 8 the score is -39660030.238723025
for i is 9 the score is -40324127.966968715


In [9]:
n_topics = 8
n_iter = 2000
lda_model = lda.LDA(n_topics=n_topics, n_iter=n_iter, random_state = 0)
X_topics = lda_model.fit_transform(cvz)

In [10]:
n_top_words = 8
topic_summaries = []

topic_word = lda_model.topic_word_  # get the topic words
vocab = cvectorizer.get_feature_names()
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: network question problem learning algorithm neural machine data
Topic 1: distribution probability random function parameter normal variable likelihood
Topic 2: data feature model set classification class validation cross
Topic 3: group data analysis variable factor test measure effect
Topic 4: model regression variable linear effect data logistic fit
Topic 5: time series data model day year event number
Topic 6: data correlation matrix cluster point plot distance set
Topic 7: test sample mean interval hypothesis standard confidence size


In [12]:
tsne_model = TSNE(n_components=2, verbose=1, random_state=0)
tsne_lda = tsne_model.fit_transform(X_topics[:10000])

[t-SNE] Computing pairwise distances...
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Computed conditional probabilities for sample 1000 / 10000
[t-SNE] Computed conditional probabilities for sample 2000 / 10000
[t-SNE] Computed conditional probabilities for sample 3000 / 10000
[t-SNE] Computed conditional probabilities for sample 4000 / 10000
[t-SNE] Computed conditional probabilities for sample 5000 / 10000
[t-SNE] Computed conditional probabilities for sample 6000 / 10000
[t-SNE] Computed conditional probabilities for sample 7000 / 10000
[t-SNE] Computed conditional probabilities for sample 8000 / 10000
[t-SNE] Computed conditional probabilities for sample 9000 / 10000
[t-SNE] Computed conditional probabilities for sample 10000 / 10000
[t-SNE] Mean sigma: 0.022845
[t-SNE] Error after 100 iterations with early exaggeration: 1.425486
[t-SNE] Error after 375 iterations: 1.355540


In [13]:
#  get the main topic for each post, which we'll use to colorize them later on:
doc_topic = lda_model.doc_topic_
lda_keys = []
for i, post in enumerate(tags):
    lda_keys += [doc_topic[i].argmax()]

In [15]:
colormap = np.array([
    "#1f77b4", "#aec7e8", "#ff7f0e", "#ffbb78", "#2ca02c", 
    "#98df8a", "#d62728", "#ff9896", "#9467bd", "#c5b0d5", 
    "#8c564b", "#c49c94", "#e377c2", "#f7b6d2", "#7f7f7f", 
    "#c7c7c7", "#bcbd22", "#dbdb8d", "#17becf", "#9edae5"
])
# plot the picture and save it.
plot_lda = bp.figure(plot_width=900, plot_height=700, title="Stack Overflow (LDA)",
    tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
    x_axis_type=None, y_axis_type=None, min_border=1)

plot_lda.scatter(x=tsne_lda[:,0], y=tsne_lda[:,1], 
                 color=colormap[lda_keys][:10000], 
                 source=bp.ColumnDataSource({
                      "tags": tags[:10000], 
                      "topic_key": lda_keys[:10000]
                }))
hover = plot_lda.select(dict(type=HoverTool))
hover.tooltips={"tags": "@tags (topic: \"@topic_key)"}
save(plot_lda, filename = '/Users/boyaliu/Documents/UCD/STA 208/final project/lda8.html')

/Users/boyaliu/anaconda/lib/python3.5/site-packages/bokeh/io.py:393: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warnings.warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/Users/boyaliu/anaconda/lib/python3.5/site-packages/bokeh/io.py:398: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warnings.warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


# Word Cloud for Tags

In [9]:
from wordcloud import WordCloud

In [10]:
# create a list contain all the tags
T = tags.tolist()
tag_l = []
for i in T:
    for j in i.split():
        tag_l.append(j)

In [13]:
tag_l

['bayesian',
 'prior',
 'elicitation',
 'distributions',
 'normality',
 'software',
 'open-source',
 'distributions',
 'statistical-significance',
 'machine-learning',
 'dataset',
 'sample',
 'population',
 'teaching',
 'humor',
 'scales',
 'measurement',
 'ordinal',
 'interval',
 'likert',
 'multivariable',
 'interpolation',
 'anova',
 'chi-squared',
 'generalized-linear-model',
 'forecasting',
 'population',
 'census',
 'bayesian',
 'frequentist',
 'distributions',
 'pdf',
 'cdf',
 'modeling',
 'time-series',
 'finance',
 'software',
 'standard-deviation',
 'basic-concepts',
 'algorithms',
 'hypothesis-testing',
 'random-variable',
 'random-generation',
 'hypothesis-testing',
 't-test',
 'p-value',
 'interpretation',
 'intuition',
 'r',
 'seasonality',
 'distributions',
 'modeling',
 'poisson',
 'overdispersion',
 'correlation',
 'teaching',
 'modeling',
 'bayesian',
 'logit',
 'transportation',
 'algorithms',
 'random-variable',
 'random-generation',
 'data-visualization',
 'intuiti

In [11]:
tag_join = ' '.join(tag_l)

In [ ]:
# take relative word frequencies into account, lower max_font_size
wordcloud = WordCloud(max_font_size=40, relative_scaling=.5, background_color="white").generate(tag_join)
plt.figure()
plt.imshow(wordcloud)
plt.axis("off")
plt.show()